In [1]:
import pandas as pd

### Leitura dados posse

In [127]:
sheet_id = '1SsVygtWx_Tv6Ps5zjhxeNJcDincPk6jpse-O5aFHQWY'
sheet_name = 'DadosPosseTJ'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
df_posse = pd.read_csv(url)

C:\Users\labci\AppData\Local\Temp\ipykernel_12676\979523307.py:4: DtypeWarning: Columns (36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df_posse = pd.read_csv(url)


In [128]:
# Filtrar por imobiliário
df_posse = df_posse.query('Imobiliário == "Sim"')
len(df_posse)

55015

In [129]:
# Filtrar para RMSP (1a RAJ + )
df_posse = df_posse.query('UTA in @cod_utas')
len(df_posse)

21737

In [130]:
# Carrega dados das UTAs
utas = pd.read_csv('wkt_utas_rmsp.txt', sep='\t', usecols=['COD_UTA', 'DOM_PP'])
utas = utas.rename(columns={'COD_UTA':'UTA'})
cod_utas = utas['UTA'].to_list()

In [131]:
def classificar_sentença(sentenca):
    if not sentenca or not isinstance(sentenca, str):
        return None
        
    sentencas = list(set([i.strip() for i in sentenca.split(',')]))
    if 'Extinção' in sentencas:
        return 'Extinta'
    if 'Procedente' in sentencas and 'Improcedente' not in sentencas:
        return 'Procedente'
    
    elif 'Improcedente' in sentencas and 'Procedente' in sentencas:
        return 'Múltipla'

    elif 'Improcedente' in sentencas:
        return 'Improcedente'

    elif 'Homologação' in sentencas:
        return 'Homologação'

    else:
        return 'Sentença não identificada'

In [132]:
# Calcular novas colunas
# Calculo do ano
df_posse['Ano'] = df_posse['Data de Disponibilização'].apply(lambda x: int(x[-4:]))

# Calculo de Ano-Mês
df_posse['Ano-Mês'] = df_posse['Data de Disponibilização'].apply(lambda x: x.split('/')[-1] + '-' + x.split('/')[-2])

# Calculo da sentença
df_posse['Desfecho'] = df_posse['Sentença'].apply(classificar_sentença)

In [133]:
# Linha do tempo
export = df_posse.query('Ano >= 2017').pivot_table(index='Ano-Mês', values='Processo', aggfunc='count').reset_index()
export = export.rename(columns = {'Processo':'Processos'})

export.to_csv('posse_por_ano-mes.csv', sep=';', index=False)

In [134]:
# Mapa dos conflitos de posse RMSP
recorte = ['Procedente', 'Homologação']
export = df_posse.query('Desfecho in @recorte').pivot_table(index='UTA', values='Processo', aggfunc='count').reset_index()
export = export.merge(utas, on='UTA')
export['Processos por domicilio'] = round(export['Processo']*100000/export['DOM_PP'],0).astype(int)
export = export.rename(columns={'Processo':'Processos'})
export = export[['UTA', 'Processos', 'Processos por domicilio']]

export.to_csv('posse_por_uta.csv', sep=';', index=False)

In [136]:
# Desfecho por ano
export = df_posse.query('Ano >= 2017').pivot_table(index='Desfecho', values='Processo', columns='Ano', aggfunc='count').reset_index()
export.to_csv('posse_desfecho_por_ano.csv', sep=';', index=False)

In [160]:
# Ano etiqueta versos ano da sentença
export = df_posse.query('`Ano etiqueta` > 2013')\
    .query('Desfecho in @recorte')\
    .pivot_table(columns='Ano etiqueta' , index = 'Ano', values='Processo', aggfunc='count')\
    .fillna(0).reset_index()
#export['Ano etiqueta'] = export['Ano etiqueta'].apply(lambda x: 'Ano ' + str(x))
export #= export.melt(id_vars = ['Ano etiqueta'])

Ano etiqueta,Ano,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,2014,254.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2015,411.0,234.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2016,172.0,413.0,238.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2017,86.0,162.0,430.0,334.0,0.0,0.0,0.0,0.0,0.0
4,2018,47.0,74.0,157.0,686.0,427.0,0.0,0.0,0.0,0.0
5,2019,30.0,54.0,87.0,296.0,728.0,496.0,0.0,0.0,0.0
6,2020,19.0,20.0,38.0,107.0,218.0,666.0,324.0,0.0,0.0
7,2021,7.0,15.0,18.0,68.0,130.0,320.0,714.0,336.0,0.0
8,2022,5.0,13.0,12.0,34.0,66.0,161.0,328.0,555.0,441.0


In [161]:
export.to_csv('posse_etiqueta_vs_datasentenca.csv', sep=';', index=False)

### Leitura dados despejos

sheet_id = ''
sheet_name = ''
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
df_posse = pd.read_csv(url)